<a href="https://colab.research.google.com/github/SS-2005/Advanced-Academic-Time-Table-Generator/blob/main/AATTG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import re
import random
from collections import defaultdict, deque
import sys

class Teacher:
    def __init__(self, name, availability, subject, departments):
        self.name = name
        self.availability = [day.strip().upper() for day in availability]
        self.subject = subject
        self.departments = [dept.strip().upper() for dept in departments]
        self.total_available_days = len(self.availability)

def parse_time(time_str):
    """Parse time string into minutes since midnight"""
    try:
        time_str = time_str.strip().upper()
        if "AM" in time_str or "PM" in time_str:
            time_part, period = re.split(r'\s+', time_str, maxsplit=1)
            hours, minutes = map(int, time_part.split(':'))
            if period == "PM" and hours != 12:
                hours += 12
            if period == "AM" and hours == 12:
                hours = 0
            return hours * 60 + minutes
        else:
            hours, minutes = map(int, time_str.split(':'))
            return hours * 60 + minutes
    except Exception as e:
        print(f"Error parsing time '{time_str}': {e}")
        return None

def format_time(minutes):
    """Format minutes into HH:MM AM/PM format"""
    hours = minutes // 60
    mins = minutes % 60
    period = "AM"
    if hours >= 12:
        period = "PM"
        if hours > 12:
            hours -= 12
    if hours == 0:
        hours = 12
    return f"{hours}:{mins:02d} {period}"

def generate_time_slots(start_min, end_min, lecture_duration, break_start_min, break_duration):
    """Generate time slots considering lecture duration and break time"""
    slots = []
    current = start_min

    # Before break
    while current < break_start_min:
        slot_end = current + lecture_duration
        if slot_end <= break_start_min:
            slots.append({
                'start_min': current,
                'end_min': slot_end,
                'formatted': f"{format_time(current)} - {format_time(slot_end)}"
            })
        current = slot_end

    # Add break slot
    slots.append({
        'start_min': break_start_min,
        'end_min': break_start_min + break_duration,
        'formatted': f"{format_time(break_start_min)} - {format_time(break_start_min + break_duration)}",
        'type': 'break'
    })

    # After break
    current = break_start_min + break_duration
    while current < end_min:
        slot_end = current + lecture_duration
        if slot_end <= end_min:
            slots.append({
                'start_min': current,
                'end_min': slot_end,
                'formatted': f"{format_time(current)} - {format_time(slot_end)}"
            })
        current = slot_end

    return slots

def get_input():
    """Collect all required inputs from the user"""
    print("="*50)
    print("ACADEMIC TIMETABLE GENERATOR")
    print("="*50)

    # Time parameters
    start_time = input("Start Time (e.g., 9:30 AM): ")
    end_time = input("End Time (e.g., 5:00 PM): ")

    try:
        lecture_duration = int(input("Lecture Duration (minutes): "))
        break_duration = int(input("Break Duration (minutes): "))
    except ValueError:
        print("Error: Duration must be a valid integer")
        return None

    break_start = input("Break Start Time (e.g., 1:30 PM): ")

    # Convert times to minutes
    start_min = parse_time(start_time)
    end_min = parse_time(end_time)
    break_start_min = parse_time(break_start)

    # Validate times
    if None in [start_min, end_min, break_start_min]:
        print("Error: Invalid time format")
        return None

    if start_min >= end_min:
        print("Error: Start time must be before end time")
        return None
    if not (start_min < break_start_min < end_min):
        print("Error: Break time must be between start and end times")
        return None

    # Teacher details
    teachers = []
    print("\nEnter Teacher Details (leave name blank when finished):")
    while True:
        name = input("\nTeacher Name (e.g., Varsha Nawale (VN)): ").strip()
        if not name:
            break
        availability = input("Available Days (comma separated, e.g., MON,TUE,WED,THU,FRI): ").split(',')
        subject = input("Assigned Subject (e.g., DWM): ").strip()
        departments = input("Assigned Departments (comma separated, e.g., AIML,CS): ").split(',')

        if not all([name, availability, subject, departments]):
            print("Error: All teacher fields are required")
            continue

        teachers.append(Teacher(name, availability, subject, departments))

    if not teachers:
        print("Error: At least one teacher is required")
        return None

    # Lab details
    lab_subjects = []
    print("\nEnter Lab Details (leave subject blank when finished):")
    while True:
        subject = input("\nLab Subject (e.g., DWM): ").strip()
        if not subject:
            break
        room = input("Lab Room Number (e.g., 517): ").strip()
        teacher = input("Teacher In-Charge (optional, press Enter to skip): ").strip()
        lab_subjects.append({
            'subject': subject.upper(),
            'room': room,
            'teacher': teacher if teacher else None
        })

    # Lab days and duration input
    lab_days = []
    while True:
        lab_days_input = input("\nEnter days for lab sessions (comma separated, e.g., MON,TUE,WED,THU): ").strip().upper()
        if lab_days_input:
            lab_days = [day.strip() for day in lab_days_input.split(',')]
            valid_days = ['MON', 'TUE', 'WED', 'THU', 'FRI']
            if all(day in valid_days for day in lab_days):
                break
            print("Error: Invalid day entered. Valid days are MON, TUE, WED, THU, FRI")
        else:
            print("Error: At least one lab day is required")

    # Lab duration
    lab_duration = 0
    while True:
        try:
            lab_duration = int(input("Lab Duration (minutes): "))
            if lab_duration % lecture_duration != 0:
                print(f"Error: Lab duration must be a multiple of lecture duration ({lecture_duration} minutes)")
                continue
            break
        except ValueError:
            print("Error: Must be a valid integer")

    # Calculate number of slots needed for lab
    lab_slot_count = lab_duration // lecture_duration

    # Other parameters
    try:
        max_repeated = int(input("\nMax Repeated Lectures per Day for a Subject (e.g., 2): "))
        batch_count = int(input("Number of Batches per Department (e.g., 4): "))
    except ValueError:
        print("Error: Must be a valid integer")
        return None

    departments = input("Departments to Generate (comma separated, e.g., AIML,CS): ").split(',')
    departments = [dept.strip().upper() for dept in departments if dept.strip()]

    if not departments:
        print("Error: At least one department is required")
        return None

    return {
        'start_min': start_min,
        'end_min': end_min,
        'lecture_duration': lecture_duration,
        'break_start_min': break_start_min,
        'break_duration': break_duration,
        'teachers': teachers,
        'lab_subjects': lab_subjects,
        'lab_days': lab_days,
        'lab_duration': lab_duration,
        'lab_slot_count': lab_slot_count,
        'max_repeated': max_repeated,
        'batch_count': batch_count,
        'departments': departments
    }

def generate_tt(inputs):
    """Generate timetable for all departments"""
    # Generate time slots
    time_slots = generate_time_slots(
        inputs['start_min'],
        inputs['end_min'],
        inputs['lecture_duration'],
        inputs['break_start_min'],
        inputs['break_duration']
    )

    if not time_slots:
        print("Error: Could not generate time slots with given parameters")
        return None

    # Calculate total non-break slots
    non_break_slots = [slot for slot in time_slots if 'type' not in slot or slot['type'] != 'break']
    total_non_break = len(non_break_slots)

    # Validate lab slot count
    if inputs['lab_slot_count'] > total_non_break:
        print(f"Error: Lab duration requires {inputs['lab_slot_count']} slots but only {total_non_break} available")
        return None

    # Days of the week
    days = ['MON', 'TUE', 'WED', 'THU', 'FRI']

    # Prepare lab rotation schedule
    lab_rotation = {}
    lab_subjects_list = [lab['subject'] for lab in inputs['lab_subjects']]
    batches = [f'B{i+1}' for i in range(inputs['batch_count'])]

    # Create rotation for each lab day
    for i, day in enumerate(inputs['lab_days']):
        rotated = deque(lab_subjects_list)
        rotated.rotate(-i)
        day_labs = list(rotated)[:inputs['batch_count']]
        lab_rotation[day] = {}
        for j, batch in enumerate(batches):
            lab_info = next(
                (lab for lab in inputs['lab_subjects'] if lab['subject'] == day_labs[j]),
                {'subject': day_labs[j], 'room': '?', 'teacher': None}
            )
            lab_rotation[day][batch] = lab_info

    # Generate timetables for each department
    timetables = {}
    for department in inputs['departments']:
        dept_tt = {day: [] for day in days}

        # Get teachers for this department
        dept_teachers = [
            t for t in inputs['teachers']
            if department in t.departments
        ]

        # Check if we have teachers for the department
        if not dept_teachers:
            print(f"Warning: No teachers found for department {department}")
            timetables[department] = dept_tt
            continue

        # Weekly subject tracking for balancing
        weekly_subject_count = defaultdict(int)
        subject_teachers = defaultdict(list)
        for teacher in dept_teachers:
            subject_teachers[teacher.subject].append(teacher)

        # Track subject hours to ensure fair distribution
        subject_hours = defaultdict(int)

        # For each day, schedule lectures and labs
        for day in days:
            subject_count = defaultdict(int)
            teacher_usage = defaultdict(int)
            is_lab_day = day in inputs['lab_days']

            # Create day-specific teacher priority
            day_teachers = [t for t in dept_teachers if day in t.availability]
            day_teachers.sort(key=lambda t: t.total_available_days)  # Prioritize teachers with limited availability

            # Calculate lab start index (end of day)
            lab_start_index = total_non_break - inputs['lab_slot_count']
            non_break_counter = 0
            lab_scheduled = False

            for slot in time_slots:
                # Handle break slot
                if slot.get('type') == 'break':
                    dept_tt[day].append({
                        'type': 'break',
                        'time': slot['formatted'],
                        'label': 'BREAK'
                    })
                    continue

                time_str = slot['formatted']

                # Handle lab sessions at end of day
                if is_lab_day and non_break_counter >= lab_start_index:
                    if not lab_scheduled:
                        dept_tt[day].append({
                            'type': 'lab',
                            'time': time_str,
                            'batch_labs': lab_rotation[day]
                        })
                        lab_scheduled = True
                    else:
                        dept_tt[day].append({
                            'type': 'lab_cont',
                            'time': time_str
                        })
                    non_break_counter += 1
                    continue

                # Track non-break slots
                non_break_counter += 1

                # Find available teachers for this slot
                candidates = []
                for subject, teachers_list in subject_teachers.items():
                    # Skip if subject already maxed for the day
                    if subject_count[subject] >= inputs['max_repeated']:
                        continue

                    for teacher in teachers_list:
                        if day in teacher.availability:
                            if teacher_usage[teacher.name] < inputs['max_repeated']:
                                candidates.append(teacher)

                # Balance subjects: prioritize less scheduled subjects
                if candidates:
                    # Sort by weekly hours to balance subjects
                    candidates.sort(key=lambda t: (subject_hours[t.subject], teacher_usage[t.name]))

                    # Randomize to get variation
                    if len(candidates) > 1:
                        top_candidates = [c for c in candidates if subject_hours[c.subject] == subject_hours[candidates[0].subject]]
                        chosen_teacher = random.choice(top_candidates)
                    else:
                        chosen_teacher = candidates[0]

                    subject = chosen_teacher.subject

                    dept_tt[day].append({
                        'type': 'lecture',
                        'time': time_str,
                        'subject': subject,
                        'teacher': chosen_teacher.name
                    })
                    subject_count[subject] += 1
                    teacher_usage[chosen_teacher.name] += 1
                    subject_hours[subject] += inputs['lecture_duration'] / 60  # Track in hours
                else:
                    dept_tt[day].append({
                        'type': 'free',
                        'time': time_str,
                        'subject': 'FREE',
                        'teacher': ''
                    })

        timetables[department] = dept_tt

    return timetables

def format_table_row(time, activity, time_width=20, activity_width=50):
    """Format a row with two columns"""
    return f"{time.ljust(time_width)} {activity}"

def show_tt(timetables):
    """Display generated timetable for all departments in a table format"""
    for dept, timetable in timetables.items():
        print(f"\n{'='*70}")
        print(f"TIMETABLE FOR DEPARTMENT: {dept}")
        print(f"{'='*70}")

        days = ['MON', 'TUE', 'WED', 'THU', 'FRI']
        for day in days:
            print(f"\n{'-'*70}")
            print(f" {day}")
            print(f"{'-'*70}")
            print(format_table_row("TIME", "ACTIVITY"))
            print(f"{'-'*70}")

            for slot in timetable[day]:
                if slot['type'] == 'lecture':
                    print(format_table_row(slot['time'], f"{slot['subject']} ({slot['teacher']})"))
                elif slot['type'] == 'free':
                    print(format_table_row(slot['time'], "FREE"))
                elif slot['type'] == 'break':
                    print(format_table_row(slot['time'], "BREAK"))
                elif slot['type'] == 'lab':
                    print(format_table_row(slot['time'], "LAB SESSION"))
                    for batch, lab_info in slot['batch_labs'].items():
                        teacher_str = f" - {lab_info['teacher']}" if lab_info['teacher'] else ""
                        batch_info = f"  • {batch}: {lab_info['subject']} (Room {lab_info['room']}{teacher_str})"
                        print(format_table_row("", batch_info))
                elif slot['type'] == 'lab_cont':
                    print(format_table_row(slot['time'], "LAB CONTINUES"))
            print(f"{'-'*70}")

# Main program
def main():
    inputs = get_input()
    if not inputs:
        print("Error in input. Exiting...")
        return

    # Set seed for consistent daily variation
    random.seed(42)

    timetables = generate_tt(inputs)
    if not timetables:
        print("Failed to generate timetable")
        return

    show_tt(timetables)

In [23]:
main()

ACADEMIC TIMETABLE GENERATOR
Start Time (e.g., 9:30 AM): 9:30 AM
End Time (e.g., 5:00 PM): 5:00 PM
Lecture Duration (minutes): 60
Break Duration (minutes): 30
Break Start Time (e.g., 1:30 PM): 1:30 PM

Enter Teacher Details (leave name blank when finished):

Teacher Name (e.g., Varsha Nawale (VN)): Varsha Nawale (VN)
Available Days (comma separated, e.g., MON,TUE,WED,THU,FRI): MON,TUE,WED,THU,FRI
Assigned Subject (e.g., DWM): DWM
Assigned Departments (comma separated, e.g., AIML,CS): AIML,CS

Teacher Name (e.g., Varsha Nawale (VN)): Maya Bangar (MB)
Available Days (comma separated, e.g., MON,TUE,WED,THU,FRI): MON,TUE,WED,THU,FRI
Assigned Subject (e.g., DWM): CN
Assigned Departments (comma separated, e.g., AIML,CS): AIML,IT

Teacher Name (e.g., Varsha Nawale (VN)): Alka Purohit (AP)
Available Days (comma separated, e.g., MON,TUE,WED,THU,FRI): MON,TUE,WED,THU,FRI
Assigned Subject (e.g., DWM): BCE
Assigned Departments (comma separated, e.g., AIML,CS): AIML,IT,CIVIL

Teacher Name (e.g., Va